# Yearly average classification table

This notebook combines the model predicitons and the grid point categories to produce a table of the improvements when going from

* V15 ---> V20
* V15 ---> V20X
* V15 ---> V15X

This improvement is denoted by `delta` i.e. $\delta = \text{V20 prediction error} - \text{V15 prediction error}$

---

#### Load all the data


In [75]:
import pandas as pd
V15 = pd.read_pickle('tmp_data/V15_predictions_yearly_average.pkl')
V20 = pd.read_pickle('tmp_data/V20_predictions_yearly_average.pkl')
V20X= pd.read_pickle('tmp_data/V20X_predictions_yearly_average.pkl')
V15X= pd.read_pickle('tmp_data/V15X_predictions_yearly_average.pkl')


change_in_fields = pd.read_pickle('tmp_data/change_in_fields.pkl')
change_in_fields['latitude_join'] = round(change_in_fields.latitude_ERA,3)
change_in_fields['longitude_join'] = round(change_in_fields.longitude_ERA,3) #just used for joining due to loss of precision from Margs file

#### Join it together - V15/V20/V20X


In [76]:
V20.shape == V15.shape == V20X.shape

True

In [77]:
#Create a new df that will just hold the differences   
data = { 'latitude_ERA':     V20.latitude_ERA, 
         'longitude_ERA':    V20.longitude_ERA,
          'MODIS_LST':       V20.MODIS_LST,
          'V15_prediction':  V15.predictions,
          'V20_prediction':  V20.predictions,
          'V20X_prediction': V20X.predictions,
          'V15_error':       V15.predicion_error,
          'V20_error':       V20.predicion_error,
          'V20X_error':      V20X.predicion_error,
          'delta':           V20.predicion_error - V15.predicion_error,
          'deltaX':          V20X.predicion_error - V15.predicion_error,
          'latitude_join':    round(V20.latitude_ERA,3), 
          'longitude_join':    round(V20.longitude_ERA,3)}

df = pd.DataFrame(data)   

In [78]:
#Join and save
df_cat = pd.merge(df,change_in_fields,how='inner',on=['latitude_join', 'longitude_join'],suffixes=('', '_y')) #inner join.
df_cat = df_cat.drop(['latitude_join', 'longitude_join','latitude_ERA_y', 'longitude_ERA_y'], axis=1) #Get rid of junk columns
df_cat.to_pickle('tmp_data/clean_yearly_data.pkl')

#### Join it together - V15/V20/V20X/V15X

Note that V15X has a different shape to the other files (part of the RML pipeline, rather than original ML pipeline)

We therefore first do an inner join to just get the grid points shared by all files.


In [79]:
#Now inner join on V15X shape
df.shape[0] == V15X.shape[0]

False

In [80]:
dfcat_withV15X = pd.merge(df_cat,V15X,how='inner',on=['latitude_ERA', 'longitude_ERA']) #inner join.

In [81]:
#Create new columns
dfcat_withV15X['V15X_prediction'] = dfcat_withV15X.predictions
dfcat_withV15X['V15X_error']      = dfcat_withV15X.predicion_error
dfcat_withV15X['deltaX15']        = dfcat_withV15X.V15X_error - dfcat_withV15X.V15_error 

#Drop the V15X columns
dfcat_withV15X = dfcat_withV15X.drop(['MODIS_LST_y', 'skt_unnormalised',
       'number_of_modis_observations', 'predictions', 'predicion_bias',
       'predicion_error'], axis=1)

#Save it
dfcat_withV15X.to_pickle('tmp_data/clean_yearly_data_w_V15X.pkl')

## Create classification table

In [91]:
df= pd.read_pickle('tmp_data/clean_yearly_data.pkl') #Load the file we just created


In [108]:
df.columns

Index(['latitude_ERA', 'longitude_ERA', 'MODIS_LST', 'V15_prediction',
       'V20_prediction', 'V20X_prediction', 'V15_error', 'V20_error',
       'V20X_error', 'delta', 'deltaX', 'V15_clFr', 'V15_cvhFr', 'V15_cvlFr',
       'V15_dl', 'V15_oceanFr', 'V15_si10Fr', 'V15_z', 'V20_clFr', 'V20_cvhFr',
       'V20_cvlFr', 'V20_dl', 'V20_oceanFr', 'V20_si10Fr', 'V20_z',
       'changeAbs_V20V15_clFr', 'changeAbs_V20V15_cvhFr',
       'changeAbs_V20V15_cvlFr', 'changeAbs_V20V15_oceanFr',
       'changeAbs_V20V15_si10Fr', 'changeAbs_V20V15_dl', 'changeAbs_V20V15_z',
       'clFr_change_is_significant', 'cvhFr_change_is_significant',
       'cvlFr_change_is_significant', 'oceanFr_change_is_significant',
       'si10Fr_change_is_significant', 'dl_change_is_significant',
       'z_change_is_significant', 'bitstring'],
      dtype='object')

In [116]:
#SLC
df.query('latitude_ERA == 41.17094491970063 & longitude_ERA == -113.39999999999998')[['V15_dl','V20_dl','changeAbs_V20V15_dl','dl_change_is_significant',
                                                                                      'V15_clFr','V20_clFr','changeAbs_V20V15_clFr','clFr_change_is_significant',
                                                                                      'V15_oceanFr','V20_oceanFr','changeAbs_V20V15_oceanFr','oceanFr_change_is_significant',
                                                                                      'bitstring']]

,V15_dl,V20_dl,changeAbs_V20V15_dl,dl_change_is_significant,V15_clFr,V20_clFr,changeAbs_V20V15_clFr,clFr_change_is_significant,V15_oceanFr,V20_oceanFr,changeAbs_V20V15_oceanFr,oceanFr_change_is_significant,bitstring
101086,3.0,27.0,24.0,True,0.530101,0.0,0.530101,True,0.0,0.0,0.0,False,1001000


In [117]:
#Gurarat
df.query('latitude_ERA == 24.028095261448925 & longitude_ERA == 69.0')[['V15_dl','V20_dl','changeAbs_V20V15_dl','dl_change_is_significant',
                                                                                      'V15_clFr','V20_clFr','changeAbs_V20V15_clFr','clFr_change_is_significant',
                                                                                      'V15_oceanFr','V20_oceanFr','changeAbs_V20V15_oceanFr','oceanFr_change_is_significant',
                                                                                      'bitstring']]

,V15_dl,V20_dl,changeAbs_V20V15_dl,dl_change_is_significant,V15_clFr,V20_clFr,changeAbs_V20V15_clFr,clFr_change_is_significant,V15_oceanFr,V20_oceanFr,changeAbs_V20V15_oceanFr,oceanFr_change_is_significant,bitstring
71529,2.580078,3.755859,1.175781,True,0.59056,0.711941,0.121381,True,0.0,0.014459,0.014459,False,1001001


In [120]:
#Peru
df.query('latitude_ERA == 34.145187223212716 & longitude_ERA == 6.0800000000000125')[['V15_dl','V20_dl','changeAbs_V20V15_dl','dl_change_is_significant',
                                                                                      'V15_clFr','V20_clFr','changeAbs_V20V15_clFr','clFr_change_is_significant',
                                                                                      'V15_oceanFr','V20_oceanFr','changeAbs_V20V15_oceanFr','oceanFr_change_is_significant',
                                                                                      'bitstring']]

,V15_dl,V20_dl,changeAbs_V20V15_dl,dl_change_is_significant,V15_clFr,V20_clFr,changeAbs_V20V15_clFr,clFr_change_is_significant,V15_oceanFr,V20_oceanFr,changeAbs_V20V15_oceanFr,oceanFr_change_is_significant,bitstring
89090,20.77832,1.0,19.77832,True,0.332777,0.000018,0.33276,True,0.0,4.297544e-08,4.297544e-08,False,1001001


In [99]:



from scipy.stats import ttest_ind
import numpy as np

def significance_test(v1,v2):
    try:
        return ttest_ind(v1, v2)
    except:
        return np.nan

def significance_boolean(x):
    
    try:
        pval = x[-1]
    except:
        #Nans are not subscriptable
        return 'Insignificant'
    
    if (pval > 0.01) or np.isnan(pval): 
        return 'Insignificant'
    else:
        return 'Significant'


def create_classification_table(ds,table_type):

    if table_type == 'V20':
        q = 'delta'
        x1 = 'V20_error'
    if table_type == 'V20X':
        q = 'deltaX'
        x1 = 'V20X_error'
    if table_type == 'V15X':
        q = 'deltaX15'
        x1 = 'V15X_error'


    classification_table                                = ds.groupby('bitstring').agg(**{'Number of Pixels':pd.NamedAgg(q,'size'),'AverageDelta':pd.NamedAgg(q,'mean')})     # For each group, count number of pixels and get the average delta      
    classification_table["Percentage"]                  = 100.0* classification_table['Number of Pixels'] / sum(classification_table["Number of Pixels"])                                            # Express number of pixels as a percentage
    
    classification_table['Stats (t-statistic,p-value)'] = ds.groupby('bitstring').apply(lambda x: significance_test(x['V15_error'], x[x1]))                                     # For each group, do a ttest between the V15_errors and the V20_errors
    classification_table['Significant Change?']         = classification_table['Stats (t-statistic,p-value)'].apply(lambda x: significance_boolean(x))

    return classification_table[['Number of Pixels', 'Percentage','AverageDelta','Stats (t-statistic,p-value)','Significant Change?']].sort_values(by=['Significant Change?', 'Number of Pixels'],ascending=False)





In [102]:
table_v20 = create_classification_table(df,'V20')
table_v20X = create_classification_table(df,'V20X')


/home/k/kimpson/bin/conda/envs/workhorse/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/k/kimpson/bin/conda/envs/workhorse/lib/python3.7/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [119]:
table_v20.sort_values(by=['Significant Change?'],ascending=False).head(60) #.iloc[0:ncircles]


,Number of Pixels,Percentage,AverageDelta,"Stats (t-statistic,p-value)",Significant Change?
bitstring,,,,,
0001000,130227,86.928109,-0.017689,"(5.018260060848922, 5.217579476526685e-07)",Significant
0001001,4431,2.957746,-0.081862,"(2.7548139055604492, 0.005884540947922857)",Significant
1001000,455,0.303718,-0.741537,"(5.443607346182554, 6.720801363659759e-08)",Significant
0011010,161,0.107469,-0.443137,"(3.616956184999805, 0.00034623551938193944)",Significant
0011001,43,0.028703,0.496087,"(-3.6206744459901947, 0.00050144658748286)",Significant
0101001,14,0.009345,8.045890,"(-10.835664011871849, 3.8850642308916806e-11)",Significant
0000000,7734,5.162539,-0.052790,"(3.258149268359118, 0.001123833128654373)",Significant
1100000,3,0.002003,-0.108881,"(2.3384231763678276, 0.07951541422321837)",Insignificant
0110001,2,0.001335,3.808017,"(-1.2408311539978356, 0.3404741913611623)",Insignificant


In [104]:
table_v20X.sort_values(by=['Significant Change?'],ascending=False).head(40) #.iloc[0:ncircles]


,Number of Pixels,Percentage,AverageDelta,"Stats (t-statistic,p-value)",Significant Change?
bitstring,,,,,
0001000,130227,86.928109,-0.080458,"(23.694647053087444, 5.544414857302782e-124)",Significant
1001000,455,0.303718,-0.796804,"(5.911530600520343, 4.793108051617881e-09)",Significant
0000000,7734,5.162539,-0.102054,"(6.472857578007254, 9.906197452180438e-11)",Significant
1011010,7,0.004673,-0.827545,"(3.0575578670087897, 0.009944069115814804)",Significant
0101010,12,0.008010,-1.046300,"(2.8526701542350863, 0.009255990974625832)",Significant
0011010,161,0.107469,-0.535915,"(4.472937210259702, 1.0732077794610761e-05)",Significant
1001001,82,0.054736,-1.159389,"(2.9960406364104086, 0.0031653934969734943)",Significant
0001010,586,0.391162,-0.211361,"(2.9628231297480667, 0.0031099409344300896)",Significant
1011000,986,0.658167,-0.143102,"(5.227713778621742, 1.898018295245507e-07)",Significant


In [90]:
table_v202.sort_values(by=['Significant Change?'],ascending=False).head(60) #.iloc[0:ncircles]


,Number of Pixels,Percentage,AverageDelta,"Stats (t-statistic,p-value)",Significant Change?
bitstring,,,,,
0001000,130227,86.928109,-0.017689,"(5.018260060848922, 5.217579476526685e-07)",Significant
0001001,4431,2.957746,-0.081862,"(2.7548139055604492, 0.005884540947922857)",Significant
1001000,455,0.303718,-0.741537,"(5.443607346182554, 6.720801363659759e-08)",Significant
0011010,161,0.107469,-0.443137,"(3.616956184999805, 0.00034623551938193944)",Significant
0011001,43,0.028703,0.496087,"(-3.6206744459901947, 0.00050144658748286)",Significant
0101001,14,0.009345,8.045890,"(-10.835664011871849, 3.8850642308916806e-11)",Significant
0000000,7734,5.162539,-0.052790,"(3.258149268359118, 0.001123833128654373)",Significant
1100000,3,0.002003,-0.108881,"(2.3384231763678276, 0.07951541422321837)",Insignificant
0110001,2,0.001335,3.808017,"(-1.2408311539978356, 0.3404741913611623)",Insignificant


---


# Appendix / scratch space

In [98]:
df.query("bitstring == '1001000'")[['V15_clFr','V20_clFr','changeAbs_V20V15_clFr','V15_oceanFr','V20_oceanFr','changeAbs_V20V15_oceanFr']]

,V15_clFr,V20_clFr,changeAbs_V20V15_clFr,V15_oceanFr,V20_oceanFr,changeAbs_V20V15_oceanFr
2298,0.123762,0.022217,0.101545,0.000000,0.074007,0.074007
2524,0.158174,0.025832,0.132342,0.000000,0.077822,0.077822
2640,0.181109,0.072806,0.108303,0.000000,0.041350,0.041350
2762,0.112202,0.001153,0.111049,0.000000,0.097715,0.097715
2793,0.165186,0.054995,0.110192,0.000000,0.074293,0.074293
...,...,...,...,...,...,...
148963,0.000000,0.136491,0.136491,0.261920,0.195660,0.066260
148965,0.000000,0.101825,0.101825,0.230743,0.235218,0.004476
148966,0.000000,0.100049,0.100049,0.324126,0.359632,0.035506
149306,0.011104,0.159325,0.148221,0.000000,0.000000,0.000000


In [95]:
df.columns

Index(['latitude_ERA', 'longitude_ERA', 'MODIS_LST', 'V15_prediction',
       'V20_prediction', 'V20X_prediction', 'V15_error', 'V20_error',
       'V20X_error', 'delta', 'deltaX', 'V15_clFr', 'V15_cvhFr', 'V15_cvlFr',
       'V15_dl', 'V15_oceanFr', 'V15_si10Fr', 'V15_z', 'V20_clFr', 'V20_cvhFr',
       'V20_cvlFr', 'V20_dl', 'V20_oceanFr', 'V20_si10Fr', 'V20_z',
       'changeAbs_V20V15_clFr', 'changeAbs_V20V15_cvhFr',
       'changeAbs_V20V15_cvlFr', 'changeAbs_V20V15_oceanFr',
       'changeAbs_V20V15_si10Fr', 'changeAbs_V20V15_dl', 'changeAbs_V20V15_z',
       'clFr_change_is_significant', 'cvhFr_change_is_significant',
       'cvlFr_change_is_significant', 'oceanFr_change_is_significant',
       'si10Fr_change_is_significant', 'dl_change_is_significant',
       'z_change_is_significant', 'bitstring'],
      dtype='object')